In [1]:
from __future__ import print_function
import jupyter_core
import os
import json
import shutil
import grp
import ipywidgets as widgets
from IPython.display import HTML, display
from notebook.services.config import ConfigManager
import hublib.ui as ui

<IPython.core.display.Javascript object>

In [2]:
cdir = os.path.join(jupyter_core.paths.jupyter_config_dir(), 'custom')
ndir = os.path.join(jupyter_core.paths.jupyter_config_dir(), 'nbconfig')
css_filename = os.path.join(cdir, 'custom.css')
cjs_filename = os.path.join(cdir, 'custom.js')
edit_filename = os.path.join(ndir, 'edit.json')
cm = ConfigManager()
doc_tools_path = 'notebook-extensions-master/calysto/document-tools/main'
inboxes_dir = '/data/tools/inboxes'

In [3]:
edit_config = """{
  "Editor": {
    "codemirror_options": {
      "vimMode": %s,
      "keyMap": "%s"
    }
  }
}
"""

editor_template = """/****** AUTOMATICALLY GENERATED BY NotebookSettings **************/

/* Warning: Anything you add here will get overwritten! */

/** KEYMAP %s **/
require(["codemirror/keymap/%s", "notebook/js/cell", "base/js/namespace"],
    function(emacs_keymap, cell, Jupyter) {
        if (Jupyter.notebook !== undefined) {
            cell.Cell.options_default.cm_config.keyMap = '%s';
            var cells = Jupyter.notebook.get_cells();
            for(var c=0; c< cells.length ; c++){
                cells[c].code_mirror.setOption('keyMap', '%s');
            }
        }
    }
);
"""

dash_css = """/****** DASHBOARD BUTTONS ******/
.jupyter-dashboard-toolbar-buttons {
    visibility: hidden;
}
"""

wide_css = """/* FULL WIDTH */
.notebook_app .container {
    width:99%; !important
}

/* Prevent the edit cell highlight box from getting clipped;
 * important so that it also works when cell is in edit mode*/
div.cell.selected {
    border-left-width: 1px !important;
}
"""

main_css = """/****** AUTOMATICALLY GENERATED BY NotebookSettings **************/

/* Warning: Anything you add here will get overwritten! */

/* remove clusters tab in tree view */
.clusters_tab_link {
    visibility: hidden;
}
/* remove CellToobar Reminder */
[title="show new celltoolbar selector location"] {
    display: none;
}
"""

eqn_js = """/** EQUATION NUMBERING **/
require([
    'base/js/namespace',
    'jquery'
],   function(IPython, $) {
    "use strict";

    IPython.toolbar.add_buttons_group([
        {
            id: 'reset_numbering',
            label: 'Reset equation numbering',
            icon: 'fa-sort-numeric-asc text-primary',
            callback: function () {
                MathJax.Hub.Queue(
                    ["resetEquationNumbers", MathJax.InputJax.TeX],
                    ["PreProcess", MathJax.Hub],
                    ["Reprocess", MathJax.Hub]
                );
                $('#reset_numbering').blur();
            }
        }
    ]);
    MathJax.Hub.Config({
        TeX: { equationNumbers: { autoNumber: "AMS" } }
    });
});
"""

In [4]:
def update_css():
    global current_width, current_dash
    with open(css_filename, 'w') as f:
        f.write(main_css)
    if current_width == 'Wide':
        with open(css_filename, 'a') as f:
            f.write(wide_css)
    if current_dash == False:
        with open(css_filename, 'a') as f:
            f.write(dash_css)
    print("Please reload any open notebooks for the changes to take effect.")

def set_width(name, width):
    global current_width
    current_width = width
    update_css()

def set_editor(name, km):
    global current_eqn, current_editor
    current_editor = km

    # write edit.json
    with open(edit_filename, 'w') as f:
        f.write(edit_config % (str(km == 'vim').lower(), km))

    # write custom.js
    with open(cjs_filename, 'w') as f:
        f.write(editor_template % (km, km, km, km))

    # equation numbering
    if current_eqn:
        with open(cjs_filename, 'a') as f:
            f.write(eqn_js)

    print("Please reload any open notebooks for the changes to take effect.")

def set_eqn(name, new):
    global current_eqn, current_editor
    current_eqn = new
    set_editor(current_editor)
    print("Please reload any open notebooks for the changes to take effect.")

def set_doc_tools(name, new):
    global current_doc
    current_doc = new
    cm.update('notebook', {"load_extensions": {doc_tools_path: current_doc}})
    print("Please reload any open notebooks for the changes to take effect.")

def set_dash_tools(name, new):
    global current_dash
    current_dash = new
    update_css()

In [7]:
# create user config dirs if none exists
if not os.path.exists(cdir):
    os.system('mkdir -p ' + cdir)
if not os.path.exists(ndir):
    os.system('mkdir -p ' + ndir)

# read current values
current_editor = None
current_eqn = False
try:
    f = open(cjs_filename, 'r')
    for line in f.read().split('\n'):
        if line.startswith('/** KEYMAP '):
            current_editor = line.split()[2]
        elif line.startswith('/** EQUATION NUMBERING'):
            current_eqn = True
    f.close()
except:
    pass

if current_editor is None:
    current_editor = 'emacs'
    set_editor('', current_editor)

current_width = None
current_dash = True
try:
    f = open(css_filename, 'r')
    for line in f.read().split('\n'):
        if line.startswith("/* FULL WIDTH */"):
            current_width = 'Wide'
        if line.startswith("/****** DASH"):
            current_dash = False
    f.close()
    if current_width is None:
        current_width = 'Wide'
except:
    pass
if current_width is None:
    current_width = 'Normal'
    set_width('', current_width)

try:
    current_doc = cm.get('notebook')['load_extensions'][doc_tools_path]
except:
    cm.update('notebook', {"load_extensions": {doc_tools_path: False}})
    current_doc = False

use_inboxes = os.path.isdir(inboxes_dir)
if use_inboxes:
    indir = os.path.join(inboxes_dir, os.environ['LOGNAME'])
    if os.path.isdir(indir):
        inboxes = sorted(os.listdir(indir))
    else:
        inboxes = []
    try:
        send_list = cm.get('notebook')['Inbox']['sendlist']
    except:
        send_list = []


Please reload any open notebooks for the changes to take effect.


# Notebook Settings

## Width
Sets the default cell width for the notebook.  Selecting 'wide' will cause the notebooks to fill the width of the browser window.

In [8]:
ui.Radiobuttons(
    name='Notebook Width',
    description="",
    value=current_width,
    options=['Normal', 'Wide'],
    width='60%',
    cb=set_width
)

A Jupyter Widget

## Editor Mode
The notebook's built-in editor can emulate other popular editors.  

In [9]:
ui.Dropdown(
    name='Keymap',
    description="Editor Keymap",
    value=current_editor,
    options={"Emacs": "emacs", "Vi": "vi", "Sublime": "sublime"},
    cb=set_editor,
    width='60%'
)

A Jupyter Widget

## Dashboard Tools

<img  src="dashboard.png" style="margin: 0"/>
Enables and disables the display of the dashboard tool buttons.  You will only need
these if you are creating dashboards (tools) instead of notebooks.

In [10]:
ui.Checkbox('Dashboard Tools', 
    'Add the Dashboard Tool Icons to the Toolbar', 
    value=current_dash, 
    cb=set_dash_tools,
    width='60%'
)

A Jupyter Widget

In [11]:
# is the user in the jupyter group?
def in_jupyter():
    try:
        return grp.getgrnam('jupyter').gr_gid in os.getgroups()
    except:
        pass
    return False

## Mailboxes

A simple way to send a notebook to another nanoHUB user.

You will need an inbox to receive notebooks. At this time, this feature is in testing and you will need to submit a ticket asking for an inbox.  A staff member will authorize it and then you can add inbox folders. 

To use Mailboxes, you must a a member of the Jupyter group.  Anyone can join at https://nanohub.org/groups/jupyter.


### Your Inbox Folders

In [12]:
def add_my_inbox(newval):
    global inboxes
    newdir = [x for x in newval if x not in inboxes][0]
    idir = os.path.join(indir, newdir)
    if not os.path.isdir(idir):
        try:
            os.makedirs(idir)
        except:
            display(ui.Modal(body="Creation of folder '%s' failed" % newdir,
                  title='ERROR',
                  buttons=['OK']))
            return
    inboxes = sorted(list(set(newval)))
    my_inbox_lm.value = inboxes

def del_my_inbox(newval):
    global inboxes
    def del_cb(val):
        global inboxes
        if val == 1:
            # delete
            shutil.rmtree(idir)
            inboxes = sorted(list(set(newval)))
        else:
            my_inbox_lm.value = inboxes

    deldir = [x for x in inboxes if x not in newval][0]
    idir = os.path.join(indir, deldir)
    if os.path.isdir(idir):
        num = len(os.listdir(idir))
        txt = "Delete folder '%s'?\n" % deldir
        if num == 0:
            txt += "(folder is empty)"
        else:
            txt += "(folder contains %d files or folders)" % num
        m = ui.Modal(body=txt,
                  title='Confirm Deletion',
                  primary=1,
                  buttons=['Cancel', 'Delete'], cb=del_cb)
        display(m)

In [13]:
def update_my_inbox(name, val):
    lval = len(val)
    lin = len(inboxes)
    if lval > lin:
        add_my_inbox(val)
    elif lval < lin:
        del_my_inbox(val)

if use_inboxes:
    if os.path.isdir(indir):
        my_inbox_lm = ui.ListManager(list_text="Inbox...", value=inboxes, cb=update_my_inbox, width='60%')
        display(my_inbox_lm)
    else:
        print("Not yet enabled. Submit a ticket if needed.")
    if not in_jupyter():
        my_inbox_lm.visible=False
        print("You are not in the Jupyter group.  Join at https://nanohub.org/groups/jupyter")

<IPython.core.display.Javascript object>

A Jupyter Widget

You are not in the Jupyter group.  Join at https://nanohub.org/groups/jupyter


In [14]:
def add_inbox(name, val):
    global send_list, inbox_lm
    send_list = []
    for v in val:
        if v.find('/') < 1:
            display(ui.Modal(body='Inbox should be in form of [username]/[folder]',
                  title='ERROR',
                  buttons=['OK']))
            continue
        d = os.path.join(inboxes_dir, v)
        if os.path.isdir(d):
            send_list.append(v)
        else:
            display(ui.Modal(body='Inbox "%s" not found!' % v,
                  title='ERROR',
                  buttons=['OK']))
    send_list = sorted(send_list)
    cm.update('notebook', {"Inbox": {'sendlist': send_list}})
    inbox_lm.value = send_list

### Destination Mailboxes

Once a user creates a mailbox, they should tell you the name so you can enter it here.
These will be added as destination mailboxes under the "File/Send To" menu.

In [15]:
if use_inboxes:
    if in_jupyter:
        inbox_lm = ui.ListManager(list_text="Mailbox...", value=send_list, cb=add_inbox, width='60%')
        display(inbox_lm)
    else:
        print("You cannot use mailboxes until you join the Jupyter group.")
        print("(If you joined in the last 15 minutes, please wait a bit then retry.)")
        print("https://nanohub.org/groups/jupyter")

<IPython.core.display.Javascript object>

A Jupyter Widget

## Document Tools

<img  src="document_tools2.png" style="margin: 0"/>
If you want your notebook sections numbered and indexed, with references at the end, you should try this.  

Adds buttons to your menubar to move sections up and down, automatically number sections, generate a table of contents and generate references.

In [16]:
ui.Checkbox('Document Tools', 
    'Add the Document Tool Icons to the Toolbar', 
    value=current_doc, 
    cb=set_doc_tools,
    width='60%'
)

A Jupyter Widget

## Equation Numbering

This will add an icon <img  src="renumber.png" style="margin: 0"/>
to the menubar which will renumber equations and generate references when clicked.
See [Equation Numbering](../manual/markdown.ipynb#Equation-Numbering-and-References) in the manual for more information.

In [17]:
ui.Checkbox('Equation Numbering', 
    'Number Latex Equations', 
    value=current_eqn, 
    cb=set_eqn,
    width='60%'
)

A Jupyter Widget

In [18]:
input_form = """
<p>Clicking 'DONE' will exit this tool.  You should confirm if prompted by your browser.</p>
<hr>
<center><button id="stop_button" style="font-size: 24px;">DONE</button></center>
"""

js = """
<script type="text/Javascript">
$(document).ready(
function() {
    $("#stop_button").click(function() {
        var close_window = function () {
            /**
             * allow closing of new tabs in Chromium, impossible in FF
            */
            window.open('', '_self', '');
            window.close();
        };
        IPython.notebook.session.delete(close_window, close_window);
        });
    });
</script>
"""
display(HTML(input_form + js))